# **I Know What You Will Do: Forecasting Motor Behaviour from EEG Time Series**

**[Brainhack Rome 2025](https://brainhackrome.github.io/) - Project #3** 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/matteo-d-m/brainhack-rome-forecasting/blob/main/eeg-forecasting-notebook.ipynb)

In [ ]:
colab = False

from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

if colab:
    !pip install mne 
import mne

import torch
from torch.utils.data import Dataset, random_split

if colab:
    from google.colab import drive

# Load data

The data are stored as `.npy` files, so we read them into `NumPy` arrays

In [ ]:
if colab: 
    data_dir = Path("/content/drive/MyDrive/Brainhack/")
else:
    data_dir = Path("/home/matteo/Documents/phd/year3/brainhack-rome-data/brainhack_dataset/P1/") # insert your local directory
    
all_npy_files = list(data_dir.rglob("*.npy"))
all_data_samples = [None]*len(all_npy_files)
for file_number, file in enumerate(all_npy_files):
    file = np.load(file)
    all_data_samples[file_number] = file
    print(f"File {file_number+1} of {len(all_npy_files)} loaded successfully!")

## Check that all data samples have the expected shape

In [ ]:
expected_shape = # insert expected shape 
for sample_number, sample in enumerate(all_data_samples):
    if sample.shape != expected_shape:
        print(f"Sample {sample_number} has shape {sample.shape} instead of {expected_shape}")

## Create PyTorch dataset and check that everything went well

In [ ]:
class DatasetFromList(Dataset):
    """Class to create a PyTorch Dataset from a list of data arrays"""
    

    def __init__(self, data, normalise=False):
        """Class constructor (i.e., it actually creates the Dataset object)
        
        Parameters:
        data -- a list of data arrays (type: list[np.array])
        normalise -- whether to normalise the samples (type: bool) (default: False)
        """

        self.data = data
        self.normalise = normalise

    def __len__(self):
        """Returns the length of the created dataset.

        Usage:
        len(dataset_name)
        """

        return len(self.data)

    def __getitem__(self, index):
        """Returns the data sample located at a given index
        
        Parameters:
        index -- the index of interest (type: int)
        
        Returns: 
        sample -- the data sample located at 'index' (type: torch.tensor)
        
        Usage:
        dataset_name[index]
        """

        sample = torch.as_tensor(self.data[index])
        sample = sample.unsqueeze(0)
        if self.normalise:
            sample = torch.nn.functional.normalize(sample)
        return sample

In [ ]:
dataset = DatasetFromList(data=all_data_samples,
                          normalise=False)

print(f"The PyTorch dataset contains {len(dataset)} samples, {'as expected' if len(dataset) == len(all_data_samples) else 'unexpectedly'}")

In [ ]:
expected_shape = # insert expected shape 
for sample_number, sample in enumerate(dataset):
    if sample.shape != expected_shape:
        print(f"Sample {sample_number} has shape {sample.shape} instead of {expected_shape}")